In [135]:
# Install dependencies
!pip install -r requirements.txt

In [110]:
"""
Download chromedriver here: https://chromedriver.chromium.org/downloads
And add exe file in this folder
"""
import os
import sys
os.path.dirname(sys.executable)

'/home/lilian/scrapping-python/env/bin'

In [111]:
# Import dependencies
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd

In [112]:
# Create class for items
class Item:
    def __init__(self):
        self.name = ""
        self.damage = ""
        self.tears = ""
        self.range = ""
        
    def searchByName(self, name):
        if self.name == name:
            return True

In [147]:
array = []
def getStats(rows, typeStat):
    for row in rows:
        name = row.find_elements(By.TAG_NAME, 'td')[0].text
        desc = row.find_elements(By.TAG_NAME, 'td')[4].text.lower()
        if "+" in desc:
            descArr = desc.split(' ')
            for i in range(len(descArr)):
                if i < len(descArr) - 1 and "+" in descArr[i] and (descArr[i+1] == typeStat or descArr[i+1] == typeStat + ','):
                    item = Item()
                    item.name = name
                    if(typeStat == "damage"):
                        item.damage = desc.split(' ')[i]
                        array.append(item)
                    elif(typeStat == "tears"):
                        isExist = False
                        for existItem in array:
                            if(existItem.searchByName(name) == True):
                                existItem.tears = desc.split(' ')[i]
                                isExist = True
                        if isExist == False:
                            item.tears = desc.split(' ')[i]
                            array.append(item)
                    elif(typeStat == "range"):
                        isExist = False
                        for existItem in array:
                            if(existItem.searchByName(name) == True):
                                existItem.range = desc.split(' ')[i]
                                isExist = True
                        if isExist == False:
                            item.range = desc.split(' ')[i]
                            array.append(item)
                    break

In [148]:
def getPage(url, element, typeStat):
    # Init driver
    driver = webdriver.Chrome()

    # Get item page
    driver.get(url)

    # Wait for load page
    time.sleep(2)

    #Accept cookies
    driver.find_element(By.XPATH,'/html/body/div[6]/div/div/div[2]/div[2]').click()
    time.sleep(1)

    #Get all items
    rows = driver.find_elements(By.XPATH, element)
    getStats(rows, typeStat)
    # Close driver
    driver.close()

In [149]:
getPage("https://bindingofisaacrebirth.fandom.com/wiki/Damage", '//*[@id="mw-content-text"]/div/table[4]/tbody/tr', "damage")

In [150]:
getPage("https://bindingofisaacrebirth.fandom.com/wiki/Tears",  '//*[@id="mw-content-text"]/div/table[5]/tbody/tr', "tears")

In [151]:
getPage("https://bindingofisaacrebirth.fandom.com/wiki/Range",  '//*[@id="mw-content-text"]/div/table[4]/tbody/tr', "range")

In [152]:
arrayItem = []
for item in array:
    arrayItem.append({"name": item.name, "damage": item.damage, "tears": item.tears, "range": item.range})
df = pd.DataFrame(arrayItem)
f = pd.DataFrame(arrayItem)
df.style
df.to_csv('items.csv', index=False)  
print(df)

                name damage  tears   range
0            Abaddon   +1.5               
1          Capricorn   +0.5          +0.75
2   Cat-O-Nine-Tails     +1               
3        Dark Matter     +1               
4      Death's Touch   +1.5               
5            Godhead   +0.5               
6    Growth Hormones     +1               
7         Guillotine     +1               
8        Jesus Juice   +0.5          +0.25
9     Magic Mushroom   +0.3           +1.5
10      Odd Mushroom   +0.3   +1.7   +0.25
11         Pentagram     +1               
12     SMB Super Fan   +0.3   +0.2        
13          Synthoil     +1          +5.25
14          The Halo   +0.3   +0.2   +0.25
15          The Mark     +1               
16          The Pact   +0.5               
17  Whore of Babylon   +1.5               
18   Maw of the Void     +1               
19         Dog Tooth   +0.3               
20             Dogma     +2               
21         Glass Eye  +0.75               
22        H